# Indexación jerárquica

Hasta ahora nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas ``Series`` y ``DataFrame``, respectivamente.
A menudo es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.
Aunque Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y cuatridimensionales, un patrón mucho más común en la práctica es hacer uso de la *indización jerárquica* (también conocida como *multiindización*) para incorporar múltiples *niveles de índice* dentro de un único índice.
De este modo, los datos de mayor dimensión pueden representarse de forma compacta dentro de los conocidos objetos unidimensionales ``Series`` y bidimensionales ``DataFrame``.

En esta sección, exploraremos la creación directa de objetos ``MultiIndex``, las consideraciones a tener en cuenta a la hora de indexar, trocear y calcular estadísticas a través de datos indexados de forma múltiple, y rutinas útiles para convertir entre representaciones simples y jerárquicamente indexadas de tus datos.

Comenzaremos con las importaciones estándar:

In [2]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Empecemos por considerar cómo podríamos representar datos bidimensionales dentro de una ``Serie`` unidimensional.
Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### The bad way

Supongamos que quieres rastrear datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, podrías estar tentado a usar simplemente tuplas de Python como claves:

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, puede indexar o trocear directamente las series basándose en este índice múltiple:

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la comodidad termina ahí. Por ejemplo, si necesitas seleccionar todos los valores a partir de 2010, tendrás que hacer algunas operaciones complicadas (y potencialmente lentas) para conseguirlo:

In [5]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que tanto nos gusta en Pandas.

### The Better Way: Pandas MultiIndex
Afortunadamente, Pandas proporciona una forma mejor.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo ``MultiIndex`` de Pandas nos proporciona el tipo de operaciones que deseamos tener.
Podemos crear un multiíndice a partir de las tuplas de la siguiente manera:

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Observe que ``MultiIndex`` contiene varios *niveles* de indexación, en este caso, los nombres de los estados y los años, así como varias *etiquetas* para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este ``MultiIndex``, veremos la representación jerárquica de los datos:

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí, las dos primeras columnas de la representación ``Series`` muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos..

Observe que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea situada encima.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, podemos simplemente utilizar la notación de Pandas slicing:

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

El resultado es un array indexado individualmente sólo con las claves que nos interesan.
Esta sintaxis es mucho más cómoda (¡y la operación es mucho más eficiente!) que la solución casera de indexación múltiple basada en tuplas con la que empezamos.
A continuación analizaremos este tipo de operaciones de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puedes notar algo más aquí: fácilmente podríamos haber almacenado los mismos datos usando un simple ``DataFrame`` con etiquetas de índice y columna.
De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente una ``Serie`` con índices múltiples en un ``DataFrame`` con índices convencionales:

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturalmente, el método ``stack()`` proporciona la operación contraria:

In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Al ver esto, puede que te preguntes por qué nos molestamos en utilizar la indexación jerárquica.
La razón es sencilla: al igual que hemos podido utilizar la indexación múltiple para representar datos bidimensionales dentro de una ``Serie`` unidimensional, también podemos utilizarla para representar datos de tres o más dimensiones en una ``Serie`` o un ``DataFrame``.
Cada nivel extra en un multiíndice representa una dimensión extra de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer añadir otra columna de datos demográficos para cada estado en cada año (digamos, población menor de 18 años); con un ``MultiIndex`` esto es tan fácil como añadir otra columna al ``DataFrame``:

In [11]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Además, todas las ufuncs y otras funcionalidades discutidas en [Operating on Data in Pandas](3_Operations-in-Pandas.ipynb) funcionan también con índices jerárquicos.

Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [12]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión.

## Métodos de creación de MultiIndex

La forma más directa de construir una ``Serie`` o un ``DataFrame`` multiíndice es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo

In [13]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.084038  0.161933
  2  0.835068  0.348316
b 1  0.000156  0.146545
  2  0.877065  0.818352

El trabajo de creación del ``MultiIndex`` se realiza en segundo plano.

Del mismo modo, si pasas un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un ``MultiIndex`` por defecto:

In [14]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; veremos un par de estos métodos aquí.

### Constructores explícitos MultiIndex

Para una mayor flexibilidad en la construcción del índice, puedes utilizar los métodos constructores de la clase ``pd.MultiIndex``.
Por ejemplo, como hicimos antes, puedes construir el ``MultiIndex`` a partir de una simple lista de arrays con los valores del índice dentro de cada nivel:

In [15]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Se puede construir a partir de una lista de tuplas que den los valores de índice múltiple de cada punto:

In [16]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Incluso se puede construir a partir de un producto cartesiano de índices simples:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

Del mismo modo, puedes construir el ``MultiIndex`` directamente utilizando su codificación interna pasando ``levels`` (una lista de listas que contienen los valores de índice disponibles para cada nivel) y ``codes`` (una lista de listas que hacen referencia a estas etiquetas):

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Cualquiera de estos objetos puede pasarse como argumento ``index`` al crear una ``Series`` o un ``Dataframe``, o pasarse al método ``reindex`` de una ``Series`` o un ``DataFrame`` ya existentes.

### Nombres de niveles MultiIndex

A veces es conveniente nombrar los niveles del ``MultiIndex``.
Esto se puede conseguir pasando el argumento ``names`` a cualquiera de los constructores de ``MultiIndex``, o estableciendo el atributo ``names`` del índice a posteriori:

In [18]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Con conjuntos de datos más complejos, puede ser una forma útil de seguir el significado de varios valores de índice.

### MultiIndex para columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas, y al igual que las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.
Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):

In [19]:
# índices jerárquicos y columnas
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# simular algunos datos
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# crear el DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      24.0  37.5  31.0  39.3  44.0  35.4
     2      27.0  37.5  58.0  36.7  30.0  37.9
2014 1      38.0  37.2  43.0  36.7  52.0  37.0
     2      35.0  37.2  37.0  37.3  38.0  35.5

En este caso, la indexación múltiple de filas y columnas puede resultar *muy* útil.
Se trata fundamentalmente de datos cuatridimensionales, en los que las dimensiones son el sujeto, el tipo de medición, el año y el número de visita.
De este modo podemos, por ejemplo, indexar la columna superior por el nombre de la persona y obtener un ``DataFrame`` completo que contenga sólo la información de esa persona:

In [20]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      31.0  39.3
     2      58.0  36.7
2014 1      43.0  36.7
     2      37.0  37.3

En el caso de registros complicados que contengan múltiples mediciones etiquetadas a lo largo de múltiples tiempos para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede resultar extremadamente cómodo.

## Indexación y segmentación de un MultiIndex

Indexar y rebanar en un ``MultiIndex`` está diseñado para ser intuitivo, y ayuda si piensas en los índices como dimensiones añadidas.
Primero veremos la indexación de ``Series`` multiíndices, y después la de ``DataFrame`` multiíndices.

### Multiply indexed Series

Consideremos la ``Serie`` de poblaciones estatales de índice múltiple que vimos anteriormente:

In [21]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Podemos acceder a elementos individuales indexando con varios términos:

In [22]:
pop['California', 2000]

33871648

El ``MultiIndex`` también admite la *indización parcial*, es decir, la indización de sólo uno de los niveles del índice.
El resultado es otra ``Serie``, que mantiene los índices de los niveles inferiores:

In [23]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

El corte parcial también está disponible, siempre que el ``MultiIndex``:

In [24]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Con índices ordenados, se puede realizar una indexación parcial en niveles inferiores pasando una rebanada vacía en el primer índice:

In [25]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

También funcionan otros tipos de indexación y selección (tratados en [Indexación y selección de datos](2_Data-Indexing-and-Selection.ipynb)); por ejemplo, la selección basada en máscaras booleanas:

In [26]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

La selección basada en una indexación elegante también funciona:

In [27]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

Un ``DataFrame`` con índices múltiples se comporta de forma similar.
Consideremos nuestro ``DataFrame`` médico de juguete de antes:

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      24.0  37.5  31.0  39.3  44.0  35.4
     2      27.0  37.5  58.0  36.7  30.0  37.9
2014 1      38.0  37.2  43.0  36.7  52.0  37.0
     2      35.0  37.2  37.0  37.3  38.0  35.5

Recuerde que las columnas son primarias en un ``DataFrame``, y que la sintaxis utilizada para las ``Series`` con índices múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de la frecuencia cardíaca de Guido con una simple operación:

In [29]:
health_data['Guido', 'HR']

year  visit
2013  1        31.0
      2        58.0
2014  1        43.0
      2        37.0
Name: (Guido, HR), dtype: float64

Además, como en el caso del índice único, podemos utilizar los indexadores ``loc``, ``iloc`` y ``ix`` introducidos en [Indexación y selección de datos](03.02-Indexación y selección de datos.ipynb). Por ejemplo:

In [30]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      24.0  37.5
     2      27.0  37.5

Estos indexadores proporcionan una vista similar a un array de los datos bidimensionales subyacentes, pero a cada índice individual de ``loc`` o ``iloc`` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [31]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        24.0
      2        27.0
2014  1        38.0
      2        35.0
Name: (Bob, HR), dtype: float64

Trabajar con trozos dentro de estas tuplas de índice no es especialmente cómodo; intentar crear un trozo dentro de una tupla provocará un error de sintaxis:

In [32]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (3311942670.py, line 1)

Se podría evitar esto construyendo la porción deseada explícitamente usando la función ``slice()`` de Python, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,24.0,31.0,44.0
2014,1,38.0,43.0,52.0


Hay muchas formas de interactuar con los datos de las ``Series`` y los ``DataFrame`` multiíndices, y como ocurre con muchas de las herramientas de este libro, la mejor forma de familiarizarse con ellas es probarlas.

## Rearranging Multi-Indices

Una de las claves para trabajar con datos de índices múltiples es saber cómo transformar los datos de forma eficaz.
Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero la reorganizarán a los efectos de diversos cálculos.
Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente el reordenamiento de datos entre índices jerárquicos y columnas, y las exploraremos aquí.

### Índices ordenados y no ordenados

Antes mencionamos brevemente una advertencia, pero deberíamos enfatizarla más aquí.
*Muchas de las operaciones de corte ``MultiIndex`` fallarán si el índice no está ordenado.*
Veámoslo aquí.

Empezaremos creando unos simples datos multiíndice en los que los índices *no están lexográficamente ordenados*:

In [34]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.957649
      2      0.350606
c     1      0.340630
      2      0.653829
b     1      0.144664
      2      0.993171
dtype: float64

Si intentamos tomar una porción parcial de este índice, se producirá un error:

In [35]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Aunque no queda del todo claro en el mensaje de error, esto se debe a que el MultiIndex no está ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles del ``MultiIndex`` estén ordenados (es decir, lexográficamente).
Pandas proporciona una serie de rutinas para realizar este tipo de ordenación; algunos ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` del ``DataFrame``.
Aquí usaremos el más simple, ``sort_index()``:

In [36]:
data = data.sort_index()
data

char  int
a     1      0.957649
      2      0.350606
b     1      0.144664
      2      0.993171
c     1      0.340630
      2      0.653829
dtype: float64

Con el índice ordenado de esta forma, el corte parcial funcionará como se espera:

In [37]:
data['a':'b']

char  int
a     1      0.957649
      2      0.350606
b     1      0.144664
      2      0.993171
dtype: float64

### Índices de apilamiento y desapilamiento

Como vimos brevemente antes, es posible convertir un conjunto de datos de un multiíndice apilado a una representación bidimensional simple, especificando opcionalmente el nivel a utilizar:

In [38]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [39]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Lo contrario de ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [40]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; esto se puede conseguir con el método ``reset_index``.
Al llamar a este método en el diccionario de población se obtendrá un ``DataFrame`` con una columna *state* y *year* que contendrá la información que antes estaba en el índice.
Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [41]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada en bruto tienen este aspecto y es útil construir un ``MultiIndex`` a partir de los valores de las columnas.
Esto se puede hacer con el método ``set_index`` del ``DataFrame``, que devuelve un ``DataFrame`` con índice múltiple:

In [42]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando me encuentro con conjuntos de datos del mundo real.

## Agregaciones de datos en multiíndices

Hemos visto anteriormente que Pandas tiene métodos de agregación de datos incorporados, como ``mean()``, ``sum()``, y ``max()``.
Para datos indexados jerárquicamente, se les puede pasar un parámetro ``level`` que controla sobre qué subconjunto de datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [43]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      24.0  37.5  31.0  39.3  44.0  35.4
     2      27.0  37.5  58.0  36.7  30.0  37.9
2014 1      38.0  37.2  43.0  36.7  52.0  37.0
     2      35.0  37.2  37.0  37.3  38.0  35.5

Tal vez nos gustaría calcular la media de las mediciones en las dos visitas de cada año. Podemos hacerlo nombrando el nivel del índice que nos gustaría explorar, en este caso el año:

In [44]:
data_mean = health_data.groupby('year').mean()
data_mean

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
year                                        
2013     25.5  37.5  44.5  38.0  37.0  36.65
2014     36.5  37.2  40.0  37.0  45.0  36.25

Haciendo uso de la palabra clave ``axis``, también podemos obtener la media entre los niveles de las columnas:

In [45]:
data_mean.T.groupby('type').mean()

year,2013,2014
type,,
HR,35.666667,40.500000
Temp,37.383333,36.816667


Así, en dos líneas, hemos podido encontrar la media de la frecuencia cardiaca y la temperatura medidas entre todos los sujetos en todas las visitas de cada año.
Esta sintaxis es en realidad un atajo a la funcionalidad ``GroupBy``, de la que hablaremos en [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb).
Aunque éste es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## Aside: Datos de panel

Pandas tiene algunas otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos ``pd.Panel`` y ``pd.Panel4D``.
Se pueden considerar, respectivamente, como generalizaciones tridimensionales y cuatridimensionales de las estructuras (unidimensionales) ``Series`` y (bidimensionales) ``DataFrame``.
Una vez familiarizado con la indexación y manipulación de datos en ``Series`` y ``DataFrame``, ``Panel`` y ``Panel4D`` son relativamente sencillos de utilizar.
En particular, los indexadores ``ix``, ``loc`` y ``iloc`` discutidos en [Data Indexing and Selection](2_Data-Indexing-and-Selection.ipynb) se extienden fácilmente a estas estructuras de mayor dimensión.

En este texto no trataremos más estas estructuras de panel, ya que en la mayoría de los casos he comprobado que la multiindización es una representación más útil y conceptualmente más sencilla para los datos de mayor dimensión.
Además, los datos de panel son fundamentalmente una representación de datos densos, mientras que la multiindización es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede resultar muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para aplicaciones especializadas ocasionales, estas estructuras pueden resultar útiles.
Si desea obtener más información sobre las estructuras ``Panel`` y ``Panel4D``, consulte las referencias que aparecen en [Recursos adicionales](13_Further-Resources.ipynb).